In [6]:
import time, os, sys
import napari
import pandas
from napari_skimage_regionprops import regionprops, add_table, get_table
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.pyplot import imread
import skimage.io
import cellpose
from skimage import data,io,filters # to use different filter on your image
from cellpose import utils, io, plot, models
import tkinter as Tk
from tkinter import* #generate dialogbox
from tkinter import filedialog
from tkinter import messagebox
import glob
from skimage.io import imsave
import cv2 as cv

#create dialogbox to select the Image data folder
root=Tk()

#setting the folder image directory
myDirectory=filedialog.askdirectory()
root.update()
path=myDirectory
path=os.path.realpath(path)
root.destroy()

time.sleep(0.5)
cells_diam=0

#setting the cell diameter size
root=Tk()
# setting the windows size
root.geometry("500x400")

#declairing the string variable
cells_diam_var = StringVar()

#defining a function that will get the cells_diam

def submit():
    global cells_diam
    cells_diam=cells_diam_var.get()
    print("the news diameter is: " + cells_diam)
    root.destroy()
    

#creating the Labelsname
name_label = Label(root, text= "cells diameter")
# creating a entry  for input
name_entry= Entry(root,textvariable = cells_diam_var)

# creating a button using the widget
# Button that will call the submit function
sub_btn=Button(root,text = 'Submit', command = submit)

# placing the label and entry in
# the required position using grid method
name_label.grid(row=3,column=0)
name_entry.grid(row=3,column=1)
sub_btn.grid(row=5,column=1)

# performing an infinite loop
# for the window to display
root.mainloop()

#open each Image and analyse 
path=os.path.realpath(path)
img=cv.imread(path)


cells_diam=int(cells_diam)
print(cells_diam)

Impath = glob.glob(path+"/*.tif")
img = []
for img in Impath:
    imgs =cv.imread(img,0)
    name=os.path.basename(img)
    name = os.path.splitext(name)[0]
    print(name)
    
    resultpath =os.path.join(path, "result",name)
    if len(os.listdir(resultpath))==0:
        # model_type='cyto' or model_type='nuclei'
        model = models.Cellpose(model_type='cyto2')
    
       # define CHANNELS to run segementation on
       # grayscale=0, R=1, G=2, B=3
       # channels = [cytoplasm, nucleus]
       # if NUCLEUS channel does not exist, set the second channel to 0
        channels = [0,0]
    
        # if diameter is set to None, the size of the cells is estimated on a per image basis
        # you can set the average cell `diameter` in pixels yourself (recommended)
        # diameter can be a list or a single number for all images

        masks, flows, styles, diams = model.eval(imgs, diameter=cells_diam, channels=channels)
    

        fig = plt.figure(figsize=(12,5))
        plot.show_segmentation(fig, imgs, masks,flows[0],channels=channels)
        plt.tight_layout()
        plt.show()
    

        viewer = napari.view_image(imgs)
        viewer.add_labels(masks)

        regionprops(
            viewer.layers[0],
            viewer.layers[1],
            viewer,
            size=True,
            shape=True,
            position=True
        )
    
       
        viewer.layers.save(resultpath)
    
    
        labels = viewer.layers[1]
        table = get_table(labels, viewer)
        ResultTable=pandas.DataFrame(table.get_content())
        df = ResultTable
        FinalResult=df[(df['bbox-0']>0) & (df['bbox-1']>0) & (df['bbox-2']>0) & (df['bbox-3']>0)] 
        FinalResult.to_csv(resultpath+"/"+name+".csv")
        FinalResult
    
        viewer = napari.Viewer()
        viewer.close()
        #image = viewer.layers['imgs']
        #imsave = ('result.tif',image)
    else:
      messagebox.showinfo("Error Message","the result folder is not empty,please delete files and try again")
    

the news diameter is: 30
30
Kamar-1


In [2]:
os.listdir(resultpath)



['imgs.tif', 'Kamar-1.csv', 'masks.tif']

Display the Table Result 

In [4]:
labels = viewer.layers[1]
table = get_table(labels, viewer)
ResultTable=pandas.DataFrame(table.get_content())
ResultTable

,label,area,bbox_area,convex_area,equivalent_diameter,max_intensity,mean_intensity,min_intensity,solidity,extent,...,eccentricity,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,weighted_centroid-0,weighted_centroid-1,standard_deviation_intensity
0,1,197,270,207,15.837556,7867,4916.248731,1938,0.951691,0.729630,...,0.952901,3.116751,173.375635,0,159,9,189,3.439968,173.746551,1435.853931
1,2,508,779,554,25.432375,9406,4240.958661,1177,0.916968,0.652118,...,0.895443,7.163386,226.285433,0,207,19,248,7.860650,227.542863,2032.140248
2,3,2513,3400,2594,56.565457,14793,3776.359331,1109,0.968774,0.739118,...,0.726798,52.124950,89.119777,20,64,88,114,52.741483,89.818812,3112.890126
3,4,2099,2881,2218,51.696516,11458,4139.880896,1132,0.946348,0.728566,...,0.818795,69.270605,127.843259,35,108,102,151,70.438803,128.020433,2811.443520
4,5,2132,2772,2192,52.101312,11857,3921.667917,1131,0.972628,0.769120,...,0.784778,82.742026,53.233114,50,33,116,75,83.933429,53.212988,2782.254281
5,6,1697,2385,1760,46.483196,17091,3989.514437,1153,0.964205,0.711530,...,0.623239,127.215675,176.571597,101,157,154,202,126.438536,174.311293,3169.383142
6,7,2231,3604,2333,53.297255,18553,4176.111161,1158,0.956279,0.619034,...,0.871849,136.216495,125.163604,103,99,171,152,138.472069,126.818211,3220.655289
7,8,1814,2440,1920,48.058886,16223,4841.397464,1193,0.944792,0.743443,...,0.782292,151.242558,34.190739,122,16,183,56,150.990438,34.884381,3889.974921
8,9,2537,4056,2655,56.834925,17754,4685.039022,1176,0.955556,0.625493,...,0.903392,170.975168,100.566417,132,75,210,127,171.155597,99.458012,3815.491058
9,10,1155,1974,1276,38.348294,13204,5418.661472,1744,0.905172,0.585106,...,0.655230,160.151515,191.280519,139,164,181,211,161.783183,192.519526,3142.370169


In [11]:
df = ResultTable
FinalResult=df[(df['bbox-0']>0) & (df['bbox-1']>0) & (df['bbox-2']>0) & (df['bbox-3']>0)] 
FinalResult

,label,area,bbox_area,convex_area,equivalent_diameter,max_intensity,mean_intensity,min_intensity,solidity,extent,...,eccentricity,centroid-0,centroid-1,bbox-0,bbox-1,bbox-2,bbox-3,weighted_centroid-0,weighted_centroid-1,standard_deviation_intensity
2,3,2513,3400,2594,56.565457,14793,3776.359331,1109,0.968774,0.739118,...,0.726798,52.124950,89.119777,20,64,88,114,52.741483,89.818812,3112.890126
3,4,2099,2881,2218,51.696516,11458,4139.880896,1132,0.946348,0.728566,...,0.818795,69.270605,127.843259,35,108,102,151,70.438803,128.020433,2811.443520
4,5,2132,2772,2192,52.101312,11857,3921.667917,1131,0.972628,0.769120,...,0.784778,82.742026,53.233114,50,33,116,75,83.933429,53.212988,2782.254281
5,6,1697,2385,1760,46.483196,17091,3989.514437,1153,0.964205,0.711530,...,0.623239,127.215675,176.571597,101,157,154,202,126.438536,174.311293,3169.383142
6,7,2231,3604,2333,53.297255,18553,4176.111161,1158,0.956279,0.619034,...,0.871849,136.216495,125.163604,103,99,171,152,138.472069,126.818211,3220.655289
7,8,1814,2440,1920,48.058886,16223,4841.397464,1193,0.944792,0.743443,...,0.782292,151.242558,34.190739,122,16,183,56,150.990438,34.884381,3889.974921
8,9,2537,4056,2655,56.834925,17754,4685.039022,1176,0.955556,0.625493,...,0.903392,170.975168,100.566417,132,75,210,127,171.155597,99.458012,3815.491058
9,10,1155,1974,1276,38.348294,13204,5418.661472,1744,0.905172,0.585106,...,0.655230,160.151515,191.280519,139,164,181,211,161.783183,192.519526,3142.370169
10,11,1301,2068,1345,40.699934,19686,7086.386626,1638,0.967286,0.629110,...,0.853691,162.989239,230.383551,139,209,186,253,163.614013,230.944735,3610.314736
11,12,2034,2806,2089,50.889775,17492,4309.404130,1146,0.973672,0.724875,...,0.740723,200.981318,71.901180,169,50,230,96,200.065674,71.711363,3636.276890


In [14]:
path = 'C:/Users/grandgie/Desktop/image users/Kamar.csv'
FinalResult.to_csv(path)